<a href="https://colab.research.google.com/github/jairzinhos/WebScraping/blob/main/Webscraping_for_dynamic_web_page_%22ElEspectador%22_newspaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Webscraping using simple tools in python (not sophisticated libraries). Take care of the structure of the sites so as to change the configuration as desired.

 Premmiun content for 'El Espectador'
Using the basic tools we can scrape a difficult dynamic web structure. It's so important to analyze the structure of the web page we trying to scrape
Analyzing html and javascript format into it so as to do a good job.
We need basic tools like BeautifulSoup, requests, and json
Url used: "https://www.elespectador.com/economia/la-reforma-pensional-empieza-a-caminar-en-que-va-la-discusion/"

In [2]:
url= input(f'Please enter the complete url address of the article')

Please enter the complete url address of the articlehttps://www.elespectador.com/economia/la-reforma-pensional-empieza-a-caminar-en-que-va-la-discusion/


In [3]:
import requests

In [4]:
r1 = requests.get(url)
coverpage = r1.content

In [5]:
import json
import re

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup1 = BeautifulSoup(coverpage, 'html5lib')

In [8]:
#We get a soup with the information needed, however we must identify the key html part where is placed our wanted data.
type(soup1)

bs4.BeautifulSoup

In [9]:
#This part concerns the html part we are looking for, but we'll have some troubles into it, because we'll find a javascript structure containing our goal data
coverpage_news = soup1.find('script', id="fusion-metadata", type="application/javascript")
coverpage_news

<script id="fusion-metadata" type="application/javascript">window.Fusion=window.Fusion||{};Fusion.arcSite="el-espectador";Fusion.contextPath="/pf";Fusion.deployment="716";Fusion.globalContent={"_id":"6N7ZWVSZCJCABGLHDJPB7E7FIU","content_elements":[{"_id":"7TMJPXOO7ZDKZJUC7HFWTNVJ7M","type":"text","additional_properties":{"_id":1686709757463,"comments":[],"inline_comments":[]},"content":"La reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes."},{"_id":"ZNF4MFEPKRFDHEMCQGETONWOY4","additional_properties":{"fullSizeResizeUrl":"/resizer/A1HZiRO6DE7rJUXFlU4iDaga9ss=/arc-anglerfish-arc2-prod-elespectador/public/ZNF4MFEPKRFDHEMCQGETONWOY4.jpg","galleries":[],"iptc_source":"Digital Vision Vectors","iptc_title":"Contributor","keywords":["abstract","achievement","authority","busines

In [12]:
# Identifying the Javascript, there is no way to navigate into it. We have to change our strategy
coverpage_news.string

'window.Fusion=window.Fusion||{};Fusion.arcSite="el-espectador";Fusion.contextPath="/pf";Fusion.deployment="716";Fusion.globalContent={"_id":"6N7ZWVSZCJCABGLHDJPB7E7FIU","content_elements":[{"_id":"7TMJPXOO7ZDKZJUC7HFWTNVJ7M","type":"text","additional_properties":{"_id":1686709757463,"comments":[],"inline_comments":[]},"content":"La reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes."},{"_id":"ZNF4MFEPKRFDHEMCQGETONWOY4","additional_properties":{"fullSizeResizeUrl":"/resizer/A1HZiRO6DE7rJUXFlU4iDaga9ss=/arc-anglerfish-arc2-prod-elespectador/public/ZNF4MFEPKRFDHEMCQGETONWOY4.jpg","galleries":[],"iptc_source":"Digital Vision Vectors","iptc_title":"Contributor","keywords":["abstract","achievement","authority","business","business person","businessman","candidate","cartoon","

In [13]:
coverpage_newsString = coverpage_news.string

Here we have a relevant function that was obtained from "https://scrapfly.io/blog/how-to-scrape-hidden-web-data/", it's so useful because the main trouble was trying to parse the javascript part where was embedded the date we were trying to extract. This function made possible to become javascript code into json format making the extraction of content throughout json navigation. The central aspect after was knowing the pattern that follows the data inside the json format in rder to extract it systematically and an ordered way.

In [14]:
import json


def find_json_objects(text: str, decoder=json.JSONDecoder()):
    """Find JSON objects in text, and generate decoded JSON data"""
    pos = 0
    while True:
        match = text.find("{", pos)
        if match == -1:
            break
        try:
            result, index = decoder.raw_decode(text[match:])
            yield result
            pos = match + index
        except ValueError:
            pos = match + 1

In [15]:
print(list(find_json_objects(coverpage_newsString)))

[{}, {'_id': '6N7ZWVSZCJCABGLHDJPB7E7FIU', 'content_elements': [{'_id': '7TMJPXOO7ZDKZJUC7HFWTNVJ7M', 'type': 'text', 'additional_properties': {'_id': 1686709757463, 'comments': [], 'inline_comments': []}, 'content': 'La reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes.'}, {'_id': 'ZNF4MFEPKRFDHEMCQGETONWOY4', 'additional_properties': {'fullSizeResizeUrl': '/resizer/A1HZiRO6DE7rJUXFlU4iDaga9ss=/arc-anglerfish-arc2-prod-elespectador/public/ZNF4MFEPKRFDHEMCQGETONWOY4.jpg', 'galleries': [], 'iptc_source': 'Digital Vision Vectors', 'iptc_title': 'Contributor', 'keywords': ['abstract', 'achievement', 'authority', 'business', 'business person', 'businessman', 'candidate', 'cartoon', 'ceo', 'clip art', 'competition', 'concepts', 'contrasts', 'copy space', 'crowd', 'domination'

In [16]:
find_json_objects(coverpage_newsString)

<generator object find_json_objects at 0x7f0990cd5d20>

In [17]:
# Creating a list with the result got by find_json function to easily navigate into it
listCoverpageNewsString = list(find_json_objects(coverpage_newsString))
listCoverpageNewsString

[{},
 {'_id': '6N7ZWVSZCJCABGLHDJPB7E7FIU',
  'content_elements': [{'_id': '7TMJPXOO7ZDKZJUC7HFWTNVJ7M',
    'type': 'text',
    'additional_properties': {'_id': 1686709757463,
     'comments': [],
     'inline_comments': []},
    'content': 'La reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes.'},
   {'_id': 'ZNF4MFEPKRFDHEMCQGETONWOY4',
    'additional_properties': {'fullSizeResizeUrl': '/resizer/A1HZiRO6DE7rJUXFlU4iDaga9ss=/arc-anglerfish-arc2-prod-elespectador/public/ZNF4MFEPKRFDHEMCQGETONWOY4.jpg',
     'galleries': [],
     'iptc_source': 'Digital Vision Vectors',
     'iptc_title': 'Contributor',
     'keywords': ['abstract',
      'achievement',
      'authority',
      'business',
      'business person',
      'businessman',
      'candidate',
      'cartoon',


The pattern behind the list in which our data is contained. It's changing so: listCoverpageNewsString[1]['content_elements'][changing part]['content']

Ahead we'll see a trick to run through every element avoiding excepetions successfully

In [19]:
listCoverpageNewsString[1]['content_elements'][0]['content']

'La reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes.'

In [20]:
listCoverpageNewsString[1]['content_elements'][2]['content']

'Buena parte de lo que puede ser el nuevo <b>Sistema General de Pensiones</b> en Colombia quedó definido este martes, tras el primer debate de la reforma pensional en la Comisión Séptima del Senado.'

In [21]:
#The length of list containing the objective data

len(listCoverpageNewsString[1]['content_elements'])

33

We detect our Key Error identifying the pattern the list have at the time of looking for our data on an ordered way. Everything is kept in a python list through the append methodm taking into account the variation of data into the json format

In [22]:
#We detect our Key Error identifying the pattern the list have at the time of looking for our data on an ordered way. Everything is kept in a python list through
#the append methodm taking into account the variation of data into the json format
articleComplete = []
contentArt = listCoverpageNewsString[1]['content_elements']
count = 0
for microContent in contentArt[0:len(contentArt)]:
  try:
    part = listCoverpageNewsString[1]['content_elements'][count]['content']
    articleComplete.append(part)
  except KeyError:
    pass
  count = 1 + count


len(articleComplete)

32

In [23]:
#The list with the objective data
articleComplete

['La reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes.',
 'Buena parte de lo que puede ser el nuevo <b>Sistema General de Pensiones</b> en Colombia quedó definido este martes, tras el primer debate de la reforma pensional en la Comisión Séptima del Senado.',
 'La ponencia del proyecto (tras su aprobación, el pasado jueves 8 de junio) llegó a la sesión con <b>44</b> artículos concertados, <b>29</b> sin proposiciones, <b>siete</b> con acuerdo parcial, <b>12 </b>sin acuerdo (la columna vertebral de la reforma, según dijo Norma Hurtado, presidenta de la comisión) y dos nuevos, para un total de 94 artículos.',
 'Tras más de seis horas de debate, <b>fueron aprobados 79 artículos</b> en una jornada que estuvo marcada por la ausencia de los senadores del Partido Conservador y d

In [24]:
#Becoming everything into a string
articleString = ''
count = 0
for art in articleComplete[0:len(articleComplete)]:
  part = articleComplete[count]
  articleString = articleString + '\n\n'+ part
  count += 1
articleString

'\n\nLa reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes.\n\nBuena parte de lo que puede ser el nuevo <b>Sistema General de Pensiones</b> en Colombia quedó definido este martes, tras el primer debate de la reforma pensional en la Comisión Séptima del Senado.\n\nLa ponencia del proyecto (tras su aprobación, el pasado jueves 8 de junio) llegó a la sesión con <b>44</b> artículos concertados, <b>29</b> sin proposiciones, <b>siete</b> con acuerdo parcial, <b>12 </b>sin acuerdo (la columna vertebral de la reforma, según dijo Norma Hurtado, presidenta de la comisión) y dos nuevos, para un total de 94 artículos.\n\nTras más de seis horas de debate, <b>fueron aprobados 79 artículos</b> en una jornada que estuvo marcada por la ausencia de los senadores del Partido Conservador y d

In [25]:
# Getting additional information of our main soup (soup1)
coverpage_newsTitle = soup1.find('title')
title = coverpage_newsTitle.get_text()
title

'La reforma pensional empieza a caminar: ¿en qué va la discusión? | EL ESPECTADOR'

In [26]:
subtitle = soup1.find('meta')['content']
subtitle

'Umbral del pilar contributivo, monto obligatorio de cotización y otras disposiciones de los artículos aprobados hoy en el Senado.'

In [27]:
type(soup1)

bs4.BeautifulSoup

In [29]:
linkUrl = soup1.find_all('link')[2]['href']
linkUrl


'https://www.elespectador.com/economia/la-reforma-pensional-empieza-a-caminar-en-que-va-la-discusion/'

In [30]:
author= soup1.find('h3', class_="ACredit-Author").get_text()
author

'Redacción Economía'

In [31]:
date = soup1.find('div', class_="Datetime ArticleHeader-Date").get_text()
date

'14 Jun 2023  - 2:26 a.\xa0m.'

In [32]:
# Collecting all data into a string
completeArticle = title + '\n' + 'Por: '+ author + '\n' + subtitle + '\n' + linkUrl + '\n' + date + '\n' + articleString
print(f"{completeArticle : >20}")
#print(f"{'Left Aligned Text' : <20}")

La reforma pensional empieza a caminar: ¿en qué va la discusión? | EL ESPECTADOR
Por: Redacción Economía
Umbral del pilar contributivo, monto obligatorio de cotización y otras disposiciones de los artículos aprobados hoy en el Senado.
https://www.elespectador.com/economia/la-reforma-pensional-empieza-a-caminar-en-que-va-la-discusion/
14 Jun 2023  - 2:26 a. m.


La reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes.

Buena parte de lo que puede ser el nuevo <b>Sistema General de Pensiones</b> en Colombia quedó definido este martes, tras el primer debate de la reforma pensional en la Comisión Séptima del Senado.

La ponencia del proyecto (tras su aprobación, el pasado jueves 8 de junio) llegó a la sesión con <b>44</b> artículos concertados, <b>29</b> sin proposiciones, <b>s

In [33]:
type(completeArticle)

str

In [34]:
# We finally have the objective data with all the features as desired
completeArticle

'La reforma pensional empieza a caminar: ¿en qué va la discusión? | EL ESPECTADOR\nPor: Redacción Economía\nUmbral del pilar contributivo, monto obligatorio de cotización y otras disposiciones de los artículos aprobados hoy en el Senado.\nhttps://www.elespectador.com/economia/la-reforma-pensional-empieza-a-caminar-en-que-va-la-discusion/\n14 Jun 2023  - 2:26 a.\xa0m.\n\n\nLa reforma pensional culminó su primer debate y unos 15 artículos la separan de su aprobación en la Comisión Séptima del Senado. No obstante, el proyecto deberá salir adelante en plenarias. Balance de lo aprobado, lo pendiente y los puntos clave del debate de este martes.\n\nBuena parte de lo que puede ser el nuevo <b>Sistema General de Pensiones</b> en Colombia quedó definido este martes, tras el primer debate de la reforma pensional en la Comisión Séptima del Senado.\n\nLa ponencia del proyecto (tras su aprobación, el pasado jueves 8 de junio) llegó a la sesión con <b>44</b> artículos concertados, <b>29</b> sin prop

It's a little bit tricky to scrape dynamic web pages, however just using basic tools it is possible to hit that goal. Analyzing html and javascript is a key part of the procedure, also to become readable the javascript part where our objective data is embedded.

In [ ]:
type(linkUrlSoup)

bs4.element.Tag